<a href="https://colab.research.google.com/github/FacundoRo/SDD/blob/master/SDD_para_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esta DarkNet con arquitectura YOLOv3 reconoce las categorías en el archivo **SDD/coco.names** . Ahi se ve que la primera categoria es **person** que es la categoria a la que se limita a trabajar.

In [1]:
!git clone https://github.com/FacundoRo/SDD

Cloning into 'SDD'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


Descargar yolov3.weights

In [0]:
#En esta google drive se encuentra yolov3.weights
#https://drive.google.com/drive/folders/1pu6-9GNVrHmAux-nqIpPUGoK1c3uNfVy

!wget https://pjreddie.com/media/files/yolov3.weights -O yolov3.weights


--2020-05-14 03:23:24--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights       87%[================>   ] 205.79M   150KB/s    eta 2m 12s 

In [0]:
!wget http://www.robots.ox.ac.uk/~lav/Research/Projects/2009bbenfold_headpose/Datasets/TownCentreXVID.avi -O TownCentreXVID.avi

Prueba SDD_video

In [0]:
import numpy as np
import time
import cv2
import math

labelsPath = "SDD/coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

weightsPath = "yolov3.weights"
configPath = "SDD/yolov3.cfg"
cap = cv2.VideoCapture('SDD/test_video.mp4')

hasFrame, frame = cap.read()
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
#vid_writer = cv2.VideoWriter('town_out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))
vid_writer = cv2.VideoWriter('salida.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (640,360))


while cv2.waitKey(1) < 0:
    
    ret,image=cap.read()
    if( not ret ):
        break
    image=cv2.resize(image,(640,360))
    (H, W) = image.shape[:2]
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    blob = cv2.dnn.blobFromImage(image, 1 / 300.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    print("Frame Prediction Time : {:.6f} seconds".format(end - start))
    print("Equivalente FPS: {:.6f}".format(1/(end-start)))
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > 0.1 and classID == 0:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2 ))
                y = int(centerY - (height / 2 ))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.3)
    ind = []
    for i in range(0,len(classIDs)):
        if(classIDs[i]==0):
            ind.append(i)
    a = []
    b = []

    if len(idxs) > 0:
            for i in idxs.flatten():
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                a.append(x)
                b.append(y)
                
    distance=[] 
    nsd = []
    for i in range(0,len(a)-1):
        for k in range(1,len(a)):
            if(k==i):
                break
            else:
                x_dist = (a[k] - a[i])
                y_dist = (b[k] - b[i])
                d = math.sqrt(x_dist * x_dist + y_dist * y_dist)
                distance.append(d)
                if(d <=30):
                    nsd.append(i)
                    nsd.append(k)
                nsd = list(dict.fromkeys(nsd))
                print(nsd)
    color = (0, 0, 255) 
    for i in nsd:
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = "Cuidado!"
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)


    color = (0, 255, 0) 
    if len(idxs) > 0:
        for i in idxs.flatten():
            if (i in nsd):
                break
            else:
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                text = 'OK'
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)   
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    vid_writer.write(image)

vid_writer.release()
cap.release()
cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15, 16]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15, 16]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15, 16]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15, 16]
[3, 1, 7, 2, 6, 8, 5, 9, 10, 11, 12, 13, 14, 15, 16

error: ignored